## Classification : Decision Tree  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
%matplotlib inline

In [2]:
#은행 데이터
bank=pd.read_csv("bank.csv")
bank.head()

FileNotFoundError: [Errno 2] File b'bank.csv' does not exist: b'bank.csv'

### EDA

In [ ]:
bank.info()
bank.isnull().sum()

In [ ]:
bank.describe()

In [ ]:
sns.boxplot(x=bank["age"])

In [ ]:
sns.distplot(bank.age, bins=100)

In [ ]:
sns.boxplot(x=bank["duration"])

In [ ]:
bankData=bank.copy()

In [ ]:
bankData.columns

In [ ]:
bankData["job"].value_counts()

In [ ]:
bankData.groupby("job").size()

In [ ]:
bankData[(bankData["job"]=="admin.") & (bankData["deposit"]=="yes")]

In [ ]:
jobs=list(set(bankData.job.values))

In [ ]:
for job in jobs:
    print("{:15}:{:5}".format(job, len(bankData[(bankData["deposit"]=="yes") & (bankData["job"]==job)])))

In [ ]:
bankData["job"]=bankData["job"].replace(["admin.","management"],"white-collar")
bankData["job"]=bankData["job"].replace(["services","housemaid"],"pink-collar")
bankData["job"]=bankData["job"].replace(["student","self-employed","technician","entrepreneur","unknown","unemployed","retired","blue-collar"],"other")

In [ ]:
bankData["job"].value_counts()

In [ ]:
bankData["poutcome"].value_counts()

In [ ]:
bankData["poutcome"]=bankData["poutcome"].replace("other","unknown")

In [ ]:
bankData["poutcome"].value_counts()

In [ ]:
bankData["contact"].value_counts()

In [ ]:
bankData.drop("contact",axis=1, inplace=True)

In [ ]:
bankData.info()

In [ ]:
bankData["default"].value_counts()

In [ ]:
bankData["default_cat"]=np.where(bankData["default"]=="yes",1,0) #방법1
bankData["default_cat"]=bankData["default"].map({"yes":1,"no":0}) #방법2

In [ ]:
bankData.drop("default",axis=1,inplace=True)

In [ ]:
bankData

In [ ]:
bankData["housing_cat"]=np.where(bankData["housing"]=="yes",1,0)
bankData.drop("housing",axis=1,inplace=True)

In [ ]:
bankData["loan_cat"]=np.where(bankData["loan"]=="yes",1,0)
bankData.drop("loan",axis=1,inplace=True)

In [ ]:
bankData["deposit_cat"]=np.where(bankData["deposit"]=="yes",1,0)
bankData.drop("deposit",axis=1,inplace=True)

In [ ]:
bankData

In [ ]:
bankData.drop("month",axis=1,inplace=True)
bankData.drop("day",axis=1,inplace=True)

In [ ]:
bankData

In [ ]:
#bankData["pdays"]==-1를 모두 10000으로 변경
bankData["pdays"]=np.where(bankData["pdays"]==-1,10000,bankData["pdays"])

In [ ]:
bankData["recent_pdays"]=np.where(bankData["pdays"], 1/bankData["pdays"],1/bankData["pdays"] )

In [ ]:
bankData["recent_pdays"]

In [ ]:
bankData.drop("pdays",axis=1,inplace=True)

In [ ]:
bankData

In [ ]:
bankWithDummies=pd.get_dummies(data=bankData, columns=["job","marital","education","poutcome"], prefix=["job","marital","education","poutcome"])

In [ ]:
bankWithDummies

In [ ]:
bankWithDummies.plot(kind="scatter", x="age", y="balance")

In [ ]:
bankWithDummies.plot(kind="hist", x="poutcome_success", y="duration")

In [ ]:
bankWithDummies[bankData.deposit_cat==1].describe()

In [ ]:
bankWithDummies[(bankWithDummies.deposit_cat==1)&(bankWithDummies.loan_cat)&(bankWithDummies.housing_cat)]

In [ ]:
len(bankWithDummies[(bankWithDummies["deposit_cat"]==1) & (bankWithDummies["default_cat"]==1)])
#52명은 정기예금 가입 및 채무불이향도 한 경우

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x="job", y="deposit_cat", data=bankData)

In [ ]:
bankData
bankWithDummies

In [ ]:
bankwd=bankWithDummies
corr=bankwd.corr()

In [ ]:
plt.figure(figsize=(15,15))
cmap=sns.diverging_palette(220,10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, linewidth=1, square=True, annot=True, vmin= 0.2 ,vmax=0.9)

In [ ]:
corr_deposit=pd.DataFrame(corr["deposit_cat"].drop("deposit_cat"))

In [ ]:
corr_deposit["deposit_cat"].sort_values(ascending=False)
#양의 상관관계가 가장 높은 것은 durantion.poutcome_success

In [ ]:
#모델 
dropDeposit= bankwd.drop("deposit_cat",1) #입력
label=bankwd.deposit_cat

In [ ]:
dataTrain, dataTest, labelTrain, labelTest=train_test_split(dropDeposit,label, train_size=0.2, random_state=42)

In [ ]:
dt2=tree.DecisionTreeClassifier(max_depth=10)
dt2.fit(dataTrain, labelTrain)
dt2_sc_train=dt2.score(dataTrain, labelTrain)
dt2_sc_test=dt2.score(dataTest, labelTest)

In [ ]:
print("트레이닝 스코어:", dt2_sc_train)
print("테스트 스코어:", dt2_sc_test)

## 연습문제(타이타닉 데이터셋)
-생존여부(출력변수)

-decision Tree(최적 depth)

-train과 test정확도

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
%matplotlib inline

In [ ]:
train=pd.read_csv("titanic/train.csv")
test=pd.read_csv("titanic/test.csv")
train

In [ ]:
#Train/Test Null값 확인

print(train.isnull().sum()) #Age 177 , Cabin 687, Embarked 2
print(test.isnull().sum())  #Age 86 , Cabin 327, Fare 1

In [ ]:
train["Survived"].value_counts()

In [ ]:
#Age컬럼 0,1로 변환
train["Sex"]=np.where(train["Sex"]=="male",0,1)
test["Sex"]=np.where(test["Sex"]=="male",0,1)

In [ ]:
# Embarked Null 값이 들어있는 컬럼 제거
train["Embarked"].dropna(inplace=True)

In [ ]:
train["Embarked"].isnull().sum()

In [ ]:
#Pclass,Embarked 원핫인코딩
titanicDummies_Train=pd.get_dummies(data=train, columns=["Embarked","Pclass"], prefix=["Embarked","Pclass"])
titanicDummies_Train

titanicDummies_Test=pd.get_dummies(data=test, columns=["Embarked","Pclass"], prefix=["Embarked","Pclass"])
titanicDummies_Test

#### Age의 Null 값, Name 호칭의 평균나이로 대체   

In [ ]:
#호칭 추출
titanicDummies_Train["title"]=titanicDummies_Train["Name"].str.extract(" ([A-Za-z]+)\.")
titanicDummies_Train["title"].value_counts()

In [ ]:
titanicDummies_Test["title"]=titanicDummies_Test["Name"].str.extract(" ([A-Za-z]+)\.")
titanicDummies_Test["title"].value_counts()

In [ ]:
#Age가 Null인 호칭 확인
titanicDummies_Train[titanicDummies_Train["Age"].isnull()].groupby("title")["Age"].size() 

In [ ]:
titanicDummies_Test[titanicDummies_Test["Age"].isnull()].groupby("title")["Age"].size()

#### Train 데이터 Age컬럼 Null값 채우기  

In [ ]:
print(titanicDummies_Train[titanicDummies_Train["title"]=="Mr"]["Age"].mean()) #32.4
print(titanicDummies_Train[titanicDummies_Train["title"]=="Miss"]["Age"].mean()) #21.8
print(titanicDummies_Train[titanicDummies_Train["title"]=="Mrs"]["Age"].mean()) #35.9
print(titanicDummies_Train[titanicDummies_Train["title"]=="Dr"]["Age"].mean()) #42.0
print(titanicDummies_Train[titanicDummies_Train["title"]=="Master"]["Age"].mean()) #4.6

In [ ]:
titanicDummies_Train.loc[titanicDummies_Train["title"]=="Mr","Age"]=titanicDummies_Train.loc[titanicDummies_Train["title"]=="Mr","Age"].fillna(32.4)
titanicDummies_Train.loc[titanicDummies_Train["title"]=="Miss","Age"]=titanicDummies_Train.loc[titanicDummies_Train["title"]=="Miss","Age"].fillna(21.8)
titanicDummies_Train.loc[titanicDummies_Train["title"]=="Mrs","Age"]=titanicDummies_Train.loc[titanicDummies_Train["title"]=="Mrs","Age"].fillna(35.9)
titanicDummies_Train.loc[titanicDummies_Train["title"]=="Dr","Age"]=titanicDummies_Train.loc[titanicDummies_Train["title"]=="Dr","Age"].fillna(42.0)
titanicDummies_Train.loc[titanicDummies_Train["title"]=="Master","Age"]=titanicDummies_Train.loc[titanicDummies_Train["title"]=="Master","Age"].fillna(4.6)

In [ ]:
#확인
titanicDummies_Train[(titanicDummies_Train["title"]=="Dr")]["Age"].isnull().sum()

In [ ]:
titanicDummies_Train[(titanicDummies_Train["title"]=="Dr")]["Age"]

In [ ]:
print(titanicDummies_Test[titanicDummies_Test["title"]=="Master"]["Age"].mean()) #7.4
print(titanicDummies_Test[titanicDummies_Test["title"]=="Miss"]["Age"].mean()) #21.8
print(titanicDummies_Test[titanicDummies_Test["title"]=="Mr"]["Age"].mean()) #32.0
print(titanicDummies_Test[titanicDummies_Test["title"]=="Mrs"]["Age"].mean()) #38.9
print(titanicDummies_Test[titanicDummies_Test["title"]=="Ms"]["Age"].mean()) #nan

In [ ]:
titanicDummies_Test.loc[titanicDummies_Test["title"]=="Master","Age"]=titanicDummies_Test.loc[titanicDummies_Test["title"]=="Master","Age"].fillna(7.4)
titanicDummies_Test.loc[titanicDummies_Test["title"]=="Miss","Age"]=titanicDummies_Test.loc[titanicDummies_Test["title"]=="Miss","Age"].fillna(21.8)
titanicDummies_Test.loc[titanicDummies_Test["title"]=="Mr","Age"]=titanicDummies_Test.loc[titanicDummies_Test["title"]=="Mr","Age"].fillna(32.0)
titanicDummies_Test.loc[titanicDummies_Test["title"]=="Mrs","Age"]=titanicDummies_Test.loc[titanicDummies_Test["title"]=="Mrs","Age"].fillna(38.9)

In [ ]:
titanicDummies_Train[titanicDummies_Train["Age"].isnull()]

In [ ]:
titanicDummies_Test[titanicDummies_Test["Age"].isnull()]

In [ ]:
titanicDummies_Test[titanicDummies_Test["Fare"]==7.75]["Age"].mean() #28.3

In [ ]:
titanicDummies_Test.loc[titanicDummies_Test["title"]=="Ms","Age"]=titanicDummies_Test.loc[titanicDummies_Test["title"]=="Ms","Age"].fillna(28.3)

In [ ]:
#train/test 데이터에서 "승객 id, 이름 , 티켓, 케빈" 모델에 필요없는 컬럼 제거

trainX=titanicDummies_Train.drop(["PassengerId","Name","Ticket","Cabin","title"],axis=1)
trainX

testX=titanicDummies_Test.drop(["PassengerId","Name","Ticket","Cabin","title"],axis=1)
testX

In [ ]:
trainX

In [ ]:
corr=trainX.corr()

plt.figure(figsize=(12,12))
cmap=sns.diverging_palette(220,10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, linewidth=1, square=True, annot=True, vmin= 0.2 ,vmax=0.9)

In [ ]:
trainX.shape

In [ ]:
testX.shape

In [ ]:
#상관관계로 fare의 Null값 채우기.
corr=testX.corr()

plt.figure(figsize=(12,12))
cmap=sns.diverging_palette(220,10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, linewidth=1, square=True, annot=True, vmin= 0.2 ,vmax=0.9)

In [ ]:
testX[(testX["Pclass_3"]==1)&(testX["Embarked_S"]==1)]["Age"].sort_values(ascending=False) #25

In [ ]:
testX[testX.index==25]

In [ ]:
testX[testX["Fare"].isnull()] #14.5로 대체

In [ ]:
testX["Fare"]=testX["Fare"].fillna(14.5)

In [ ]:
testX["Fare"]

In [ ]:
#정답데이터
X_label=trainX["Survived"]  #분류예측값
trainX= trainX.drop("Survived",1) #입력

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 데이터 정규화
scaler = MinMaxScaler()

train_final = scaler.fit_transform(np.array(trainX))
test_final= scaler.fit_transform(np.array(testX))

## SVC 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param = {'C':[1,5,10,20,40,100],
         'gamma':[.1, .25, .5, 1]}

GS = GridSearchCV(SVC(kernel='rbf'), param, cv = 5) #최적의 param값 찾아줌.

GS.fit(train_final, X_label)

print(GS.best_params_) #C: 10, gamma: 0.5 일때
print(GS.best_score_) #0.8226711560044894


# 모델 생성
model = SVC(kernel='rbf', C=10, gamma=0.5).fit(train_final, X_label)

prediction_SVC = model.predict(test_final)
prediction_SVC

# 결과 데이터 csv 생성
sub = pd.read_csv('titanic/gender_submission.csv')
sub['Survived'] = prediction_SVC
sub.to_csv('submission_svc.csv', index=False)

#Kaggle upload result: 0.78468

## Decision Tree Model

In [ ]:
#모델생성
decision_tree =tree.DecisionTreeClassifier(max_depth=15)
decision_tree.fit(train_final, X_label)
Y_pred = decision_tree.predict(test_final)

In [ ]:
sub = pd.read_csv("titanic/gender_submission.csv")
sub["Survived"] = Y_pred
sub.to_csv("submission_titanic.csv",index=False)
pd.read_csv("submission_titanic.csv")

In [ ]:
#Train data에서 Train/Test Split해서 모델 성능평가
dataTrain,dataTest,labelTrain,labelTest = train_test_split(train_final,X_label,train_size=0.8,random_state=42)

In [ ]:
dt=tree.DecisionTreeClassifier(max_depth=10)
model = dt.fit(dataTrain,labelTrain)
dt_sc_train=dt.score(dataTrain, labelTrain)
dt_sc_test=dt.score(dataTest, labelTest)

print("트레이닝 스코어:", dt_sc_train) #0.922752808988764
print("테스트 스코어:", dt_sc_test) #0.7988826815642458

#Kaggle upload result: 0.75598